In [ ]:
x = randn(50,10)
mean(var(x,2))

In [ ]:
using Plots
gr()

In [ ]:
function compareGaussian(sxx, syy, mu, S, iS, d1=1, d2=2)
    # get 2d sub-covariance matrix
    Ss = S[[d1,d2],[d1,d2]]
    # get mean & diagonal sd
    mu1, mu2 = mu[d1], mu[d2]
    s1,  s2  = sqrt(S[d1,d1]), sqrt(S[d2,d2])

    x1min, x1max = mu1 + 2*[-s1,s1]
    x2min, x2max = mu2 + 2*[-s2,s2]

    ng    = 50 
    x     = linspace(x1min, x1max, ng)
    y     = linspace(x2min, x2max, ng)
    xgrid = repmat(x', ng, 1)
    ygrid = repmat(y, 1, ng)

    z = zeros(ng,ng)
    for i in 1:ng, j in 1:ng
        dx     = [x[i],y[j]]-[mu1,mu2]
        z[i,j] = exp(-dot(iS*dx, dx))
    end

    contour(x,y,z)
    scatter!(sxx,syy, markersize=0.1)
end

In [ ]:
include("hmc.jl")

In [ ]:
S  = [1/3 1/2; 1/2 1];
iS = [12 -6; -6 4];
mu = [0.5, 0.5]

loglik(x)     = -dot(iS*(x-mu), (x-mu))/2
gradloglik(x) = iS*(mu-x)

x0 = [0.2, 0.2]

samples = hmc(loglik, gradloglik, x0; steps=10000, stepsize=1e-1);

In [ ]:
xx = [samples[i][1] for i in 1:length(samples)]
yy = [samples[i][2] for i in 1:length(samples)]
;

In [ ]:
compareGaussian(xx, yy, mu, S, iS)

In [ ]:
println(round(mean(xx),2), " -x- ", mu[1])
println(round(mean(yy),2), " -y- ", mu[2])
println(round(std(xx), 2), " -x- ", sqrt(S[1,1]))
println(round(std(yy), 2), " -y- ", sqrt(S[2,2]))

## PMF likelihood

In [ ]:
rows  = vec(readdlm("data/rows.csv",  Int))
cols  = vec(readdlm("data/cols.csv",  Int))
rates = vec(readdlm("data/rates.csv", Int))
;

this should probably be hardcoded, there may be additionnal lines / columns ?

In [ ]:
nrows = maximum(rows)
ncols = maximum(cols)
println(nrows, " ", ncols)

In [ ]:
[sum(rates.==r) for r in unique(rates)]

In [ ]:
include("pmf_ll.jl")

In [ ]:
sr = 1.0
su = 20.0
sv = 20.0
d  = 30
(ll, gll) = pmf_ll(rows, cols, rates, nrows, ncols, sr,su,sv,d)
;

In [ ]:
utest = su*randn(d*nrows)
vtest = sv*randn(d*ncols)
;

In [ ]:
ll(vcat(utest,vtest))

In [ ]:
ll(vcat(zeros(d*nrows), zeros(d*ncols)))

In [ ]:
gll(vcat(utest, vtest))

In [ ]:
include("pmf_rmse.jl")

In [ ]:
pmf_rmse(rows, cols, rates, nrows, ncols, d, vcat(utest, vtest))

In [ ]:
samples = hmc(ll, gll, randn(299760); steps=50, burnin=5, stepsize=0.001);

In [ ]:
ss = sum(samples)/length(samples)
pmf_rmse(rows, cols, rates, nrows, ncols, d, ss)

In [ ]:
# rmse from all zero
pmf_rmse(rows, cols, rates, nrows, ncols, d, similar(ss))

In [ ]:
length(ss)